<a href="https://colab.research.google.com/github/qmaruf/pyspark/blob/main/pyspark_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget https://raw.githubusercontent.com/leonpalafox/SparkCode/master/1800.csv

--2021-01-16 15:25:11--  https://raw.githubusercontent.com/leonpalafox/SparkCode/master/1800.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62728 (61K) [text/plain]
Saving to: ‘1800.csv’

1800.csv            100%[===================>]  61.26K  --.-KB/s    in 0.01s   

2021-01-16 15:25:11 (4.51 MB/s) - ‘1800.csv’ saved [62728/62728]



In [2]:
! pip install pyspark

     |████████████████████████████████| 204.2MB 51kB/s 
     |████████████████████████████████| 204kB 46.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=c826b6a55f18b3c64e439478189420c653e75dafeea6358f832b0edfb9083815
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
from pyspark import SparkConf, SparkContext

In [4]:
conf = SparkConf().setMaster('local').setAppName('MinTemparatures')
sc = SparkContext(conf = conf)

In [5]:
sc

<SparkContext master=local appName=MinTemparatures>

In [12]:
data = sc.textFile('./1800.csv')

In [13]:
data.take(5)

['ITE00100554,18000101,TMAX,-75,,,E,',
 'ITE00100554,18000101,TMIN,-148,,,E,',
 'GM000010962,18000101,PRCP,0,,,E,',
 'EZE00100082,18000101,TMAX,-86,,,E,',
 'EZE00100082,18000101,TMIN,-135,,,E,']

In [14]:
def parselines(line):
    parts = line.split(',')
    stationID, entryType = parts[0], parts[2]
    temparature = float(parts[3])
    return (stationID, entryType, temparature)

lines = data.map(parselines)

In [15]:
lines.take(5)

[('ITE00100554', 'TMAX', -75.0),
 ('ITE00100554', 'TMIN', -148.0),
 ('GM000010962', 'PRCP', 0.0),
 ('EZE00100082', 'TMAX', -86.0),
 ('EZE00100082', 'TMIN', -135.0)]

In [16]:
minTemparature = lines.filter(lambda x: x[1] == 'TMIN')
minTemparature.take(5)

[('ITE00100554', 'TMIN', -148.0),
 ('EZE00100082', 'TMIN', -135.0),
 ('ITE00100554', 'TMIN', -125.0),
 ('EZE00100082', 'TMIN', -130.0),
 ('ITE00100554', 'TMIN', -46.0)]

In [17]:
stationTemps = minTemparature.map(lambda x: (x[0], x[2]))

In [18]:
stationTemps.take(5)

[('ITE00100554', -148.0),
 ('EZE00100082', -135.0),
 ('ITE00100554', -125.0),
 ('EZE00100082', -130.0),
 ('ITE00100554', -46.0)]

In [19]:
minTemparature = stationTemps.reduceByKey(lambda x, y: min(x, y))

In [20]:
minTemparature.take(10)

[('ITE00100554', -148.0), ('EZE00100082', -135.0)]

In [21]:
minTemparature.collect()

[('ITE00100554', -148.0), ('EZE00100082', -135.0)]

In [22]:
minTemparatureDegC = minTemparature.map(lambda x: x[1]*0.1*(9.0/5.0)+32)

In [24]:
minTemparatureDegC.take(10)

[5.359999999999999, 7.699999999999999]